# 第2回データ分析コンペティション：Home Credit Default Risk

## 背景
　今回のコンペでは 2018年にKaggleで開催された**"Home Credit Default Risk"** (https://www.kaggle.com/c/home-credit-default-risk/overview/description) に取り組んでいただきます。このコンペでは、与えられた様々な顧客データを元に各データが債務不履行になるかどうかを予測します。本来のコンペでは複数のデータセットが用意されていますが、今回はそれらのデータセットの一部を配布します。

## 目標
　顧客のデータからその顧客が債務不履行(default)になる**確率**を予測してください。データセットのTARGETカラムが目的変数、それ以外が説明変数となります。TARGETカラムが1であれば支払い困難、0であればそれ以外を表しています。

## データ
　データ分析コンペティションでは、一般的に**訓練データ**と**テストデータ**が与えられます。今回の場合、訓練データは**input/train.csv**、テストデータは**input/test.csv**となっています。訓練データとテストデータは以下のような点で異なります。  
- 訓練データ：このデータによって機械学習モデルを学習させます。
- テストデータ：このデータに対する予測を提出して、その精度を競います。   

　また、訓練データに含まれるカラムの情報については**HomeCredit_columns_description.xlsx**を参照してください。

In [4]:
# Google Drive と接続
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [25]:
import numpy as np
import pandas as pd

# csv ファイルパス，随時変更
file_path = 'drive/My Drive/Colab Notebooks/competition2/input/'

def one_hot_encoder(data, nan_as_category = True):
    original_columns = list(data.columns)
    categorical_columns = [f for f in data.columns if df[f].dtype == 'object']
    for c in categorical_columns:
        if nan_as_category:
            data[c].fillna('NaN', inplace = True)
        values = list(data[c].unique())
        for v in values:
            data[str(c) + '_' + str(v)] = (data[c] == v).astype(np.uint8)
    data.drop(categorical_columns, axis = 1, inplace = True)
    return data, [c for c in data.columns if c not in original_columns]

def application_train_test(file_path = file_path, nan_as_category = True):
  # ファイル読み込み
  df = pd.read_csv(file_path + 'train.csv')
  df_test = pd.read_csv(file_path + 'test.csv')

  df = df.append(df_test).reset_index()

  # Remove some rows with values not present in test set
  df = df[df['CODE_GENDER'] != 'XNA']
  df.drop(df[df['NAME_FAMILY_STATUS'] == 'Unknown'].index, inplace = True)

  # Categorical features with Binary encode (0 or 1; two categories)
  bin_cols = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']
  for bin_feature in bin_cols:
      df[bin_feature], _ = pd.factorize(df[bin_feature])

  # Categorical features with One-Hot encode
  cat_cols = [f for f in df.columns if df[f].dtype == 'object']
  for cat_feature in cat_cols:
      df[cat_feature], _ = pd.factorize(df[cat_feature])
  
  #df, _ = one_hot_encoder(df, nan_as_category)

  # Replace some outliers
  # https://www.kaggle.com/aantonova/797-lgbm-and-bayesian-optimization
  df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
  df.loc[df['OWN_CAR_AGE'] > 80, 'OWN_CAR_AGE'] = np.nan
  df.loc[df['REGION_RATING_CLIENT_W_CITY'] < 0, 'REGION_RATING_CLIENT_W_CITY'] = np.nan
  df.loc[df['AMT_INCOME_TOTAL'] > 1e8, 'AMT_INCOME_TOTAL'] = np.nan
  df.loc[df['AMT_REQ_CREDIT_BUREAU_QRT'] > 10, 'AMT_REQ_CREDIT_BUREAU_QRT'] = np.nan
  df.loc[df['OBS_30_CNT_SOCIAL_CIRCLE'] > 40, 'OBS_30_CNT_SOCIAL_CIRCLE'] = np.nan

  # Some simple new features (percentages)
  df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
  df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
  df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
  df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
  df['PAYMENT_PERC'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']

  # original
  return df

df = application_train_test(file_path, True)
df

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,...,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Industry: type 10,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Industry: type 12,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Trade: type 5,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_PERC
0,0,0,0.0,0,0,0,0,112500.0,755190.0,36328.5,675000.0,0.010032,-9233,-878.0,-333.0,-522,NaN,1,1,1,1,0,0,2.0,2,2.0,0,1,1,0,1,1,NaN,0.372591,NaN,0.0,0.0,0.0,0.0,-292.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.095094,0.148969,56250.0,0.322920,20.787811
1,1,1,0.0,0,0,1,0,225000.0,585000.0,16893.0,585000.0,0.008019,-20148,NaN,-4469.0,-3436,NaN,1,0,0,1,0,0,2.0,2,2.0,0,0,0,0,0,0,NaN,0.449567,0.553165,0.0,0.0,0.0,0.0,-617.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0.384615,112500.0,0.075080,34.629728
2,2,2,0.0,0,0,1,0,54000.0,334152.0,18256.5,270000.0,0.004960,-18496,-523.0,-3640.0,-2050,NaN,1,1,1,1,1,0,2.0,2,2.0,0,0,0,0,0,0,NaN,0.569503,NaN,4.0,0.0,4.0,0.0,-542.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.028276,0.161603,27000.0,0.338083,18.303180
3,3,3,0.0,0,0,1,0,67500.0,152820.0,8901.0,135000.0,0.005002,-24177,NaN,-4950.0,-3951,NaN,1,0,0,1,1,0,1.0,3,3.0,0,0,0,0,0,0,NaN,0.105235,0.767523,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0.441696,67500.0,0.131867,17.168857
4,4,4,1.0,1,0,0,0,157500.0,271066.5,21546.0,234000.0,0.006296,-10685,-697.0,-5101.0,-3226,NaN,1,1,1,1,0,0,2.0,3,3.0,0,0,0,0,1,1,0.342344,0.202490,0.669057,0.0,0.0,0.0,0.0,-1243.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.065232,0.581038,78750.0,0.136800,12.580827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232697,61495,232697,NaN,0,1,1,0,315000.0,1288350.0,37800.0,1125000.0,0.007020,-11430,-792.0,-9772.0,-2705,12.0,1,1,0,1,0,0,2.0,2,2.0,0,0,0,0,0,0,0.263678,0.018172,0.307737,0.0,0.0,0.0,0.0,-1.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.069291,0.244499,157500.0,0.120000,34.083333
232698,61496,2

In [ ]:
# ベースラインモデルの構築

# 必要なライブラリ等のインポート
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

def kfold_lightgbm(df, num_folds, debug = False):
  train_df = df[df['TARGET'].notnull()]
  test_df = df[df['TARGET'].isnull()]

  print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

  # Cross validation model
  folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)

  # Create arrays and dataframes to store results
  oof_preds = np.zeros(train_df.shape[0])
  sub_preds = np.zeros(test_df.shape[0])
  feature_importance_df = pd.DataFrame()
  feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR']]

  for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
      train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
      valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

      # LightGBM parameters found by Bayesian optimization
      # Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
      clf = lgb.LGBMClassifier(
          nthread=4,
          n_estimators=10000,
          learning_rate=0.03,
          num_leaves=34,
          colsample_bytree=0.9497036,
          subsample=0.8715623,
          max_depth=8,
          reg_alpha=0.041545473,
          reg_lambda=0.0735294,
          min_split_gain=0.0222415,
          min_child_weight=39.3259775,
          silent=-1,
          verbose=-1, )

      clf.fit(train_x, train_y, 
              eval_set=[(train_x, train_y), (valid_x, valid_y)], 
              eval_metric= 'auc',
              verbose= 200,
              early_stopping_rounds= 200)

      oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
      sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

      fold_importance_df = pd.DataFrame()
      fold_importance_df["feature"] = feats
      fold_importance_df["importance"] = clf.feature_importances_
      fold_importance_df["fold"] = n_fold + 1
      feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
      print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))

  # ファイル格納先，随時変更
  submission_file_name = 'drive/My Drive/Colab Notebooks/competition2/default_submission.csv'
  print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
  if not debug:
    test_df['TARGET'] = sub_preds
    test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)

    submission = pd.read_csv(submission_file_name)
    print(submission)

  display_importances(feature_importance_df)
  return feature_importance_df

def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances.png')

kfold_lightgbm(df, num_folds=10, debug= False)

In [14]:
submission_file_name = 'drive/My Drive/Colab Notebooks/competition2/default_submission.csv'
submission = pd.read_csv(submission_file_name)
submission['TARGET'].value_counts()

0        0.022757
1        0.151386
2        0.157697
3        0.091713
4        0.225074
           ...   
61495    0.163091
61496    0.029341
61497    0.048141
61498    0.186495
61499    0.116958
Name: TARGET, Length: 61500, dtype: float64

In [12]:
%pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=2d098e9b250fca5ed42231ae06a1c9922324fb3a61195e914e699c916e5968c4
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [ ]:
# パラメータチューニング
import pandas as pd
import lightgbm as lgb
from bayes_opt import BayesianOptimization

df = application_train_test(file_path, True)

train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
y = train_df['TARGET']

data = lgb.Dataset(data=train_df, label=y)

def parameters(
    num_iterations, 
    learning_rate, 
    num_leaves,   
    subsample, 
    max_depth, 
    reg_alpha, 
    reg_lambda, 
    min_split_gain, 
    min_child_weight, 
    ):
  params = ('application' : 'binary', 'early_stopping_round' : 100, 'metric' : 'auc')
  params['num_iterations'] = int(round(num_iterations))
  params['learning_rate'] = learning_rate
  params['num_leaves'] = int(round(num_leaves))
  params['subsample'] = max(min(subsample, 1), 0)
  params['max_depth'] = int(round(max_depth))
  params['reg_alpha'] = max(reg_alpha, 0)
  params['reg_lambda'] = max(reg_lambda, 0)
  params['min_split_gain'] = min_split_gain
  params['min_child_weight'] = min_child_weight

  cv_result = lgb.cv(params, data, nfold = 5, seed = 6, stratified = True, metrics = ['auc'])
  return max(cv_result['auc-mean'])

optimizer = BayesianOptimization(parameters, {
    'num_iterations' : (9800, 11000),
    'learning_rate': (.01, .02), 
    'num_leaves': (33, 35), 
    'subsample': (0.8, 1), 
    'max_depth': (7, 9), 
    'reg_alpha': (.03, .05), 
    'reg_lambda': (.06, .08), 
    'min_split_gain': (.01, .03),
    'min_child_weight': (38, 40)
    })
optimizer.maximize(init_points=30, n_iter=30)
print(optimizer.max)

## 評価指標
　今回のコンペティションでは、テストデータに対して予測された確率と正解データとの間の[ROC曲線下の面積](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)である**AUC**を競っていただきます。

## 提出/採点/順位
　予測は**Omnicampus**上で提出してください。提出するデータの形式は以下のようなcsvファイルとなります。
 
SK_ID_CURR|TARGET
---|---
171202|0.5
171203|0.5
171204|0.5
…|…
232699|0.5
232700|0.5
232701|0.5

　また**sample_submission.csv**がこの形式に準じた提出ファイルの例となっているので、参考にしてください。 提出ファイルがこの形式に準じていなかった場合にはスコアが-999となるので、そのような際は提出ファイルの形式を確認してみましょう。  

　提出自体の回数は無制限ですが、採点は1日に3回、以下の時間に行われます。
   - 3時
   - 15時
   - 21時

　コンペティション期間中の採点はテストデータの一部に対して行われます。これは「順位表に対する過学習」という現象を避けるためです。この採点結果に基づいてコンペティション期間中はOmnicampus上で順位表が更新されていきます。このようにコンペティション期間中に一部のデータに対する採点結果に基づいて更新される順位表は**Public Leaderboard**と呼ばれます。  

　これに対して、コンペティション終了後に行われる採点はテストデータ全体に対して行われます。最終順位はこの採点結果に基づいて確定されます。このように確定された順位表はPublic Leaderboardに対して**Private Leaderboard**と呼ばれます。

## タイムライン
開始日： 2020年6月10日 (水)  
提出期限： **2020年6月28日 (日) 23:59:59**  
順位発表： 2020年6月29日 (月)

## ルール
- **外部データ使用の禁止**  
　データ分析においては、EDAやモデルの学習など全ての段階において外部データを使用しないでください。つまり今回のコンペティションでは、input/train.csvおよびinput/test.csvのみを使用して、データ分析に取り組んでください。  

- **Hand-Labelingの禁止**  
　モデルによってではなく手作業で予測を作成することは**Hand-Labeling**と呼ばれ、ほとんどのデータ分析コンペティションでは禁止されています。今回のコンペティションでもこのルールを採用し、全てあるいは一部のテストデータに対してHand-Labelingをすることを禁止します。予測は訓練データやテストデータ以外のデータに対しても適用可能なモデルによって自動で為されるようにしてください。

- **再現性の確保**  
　可能な限り予測の再現性を確保してください。再現性を保つためには、**乱数生成においてシード値を指定すること**が必要条件です。  
 「可能な限り」という表現がなされているのは、再現性がどうしても保てない場合が存在するためです。例えば最近では、PyTorchというディープラーニングフレームワークがバージョンアップにより全体的に精度を落とすということがありました。この場合、バージョンアップ前に作成した予測をバージョンアップ後に再現するためには、バージョンダウンする必要があります。今回のコンペティションにおいては、再現性の確保のためにそこまでする必要はありません。  

- **Private Sharingの許可**  
　データ分析コンペティションでは一般的に、公平性の観点から個人的な情報共有は厳しく規制されています。多くの場合、チームの結成を規定の方法で宣言した場合のみチーム内での情報共有が許可されます。チーム外の個人的な情報共有は**Private Sharing**と呼ばれ、固く禁じられています。  
　しかし今回のコンペティションは入門者の参加するチュートリアルとしての位置付けであるため、Private Sharingの禁止は適用しません。身近に経験者がいれば積極的にアドバイスを求めても良いですし、オンラインで他の受講生と情報交換やディスカッションをしても構いません。ただしオンラインで他の受講生と情報交換やディスカッションをする際には、できるだけ個人的なやりとりではなく公開された場で行い、全ての受講生が知見を共有できるよう心がけてください。